# tiff2jpg Notebook

This notebook will help us in achieving the followings:

* It helps us in creating the different combinations of bands, indices or properties for the conversion of raster file into jpg format.
* In Brief, it generates and save the jpg file converted from tif files.

use the python3 environment in the EC2 Deep Learning AMI. Install tqdm, scikit-image, and rasterio

# Importing The Libraries

In [40]:
# import helper functions 

# add helper function location
import sys 
sys.path.append('../../functions')

from helpers import *

from importlib import reload  
reload(helpers)

<module 'helpers' from '../../functions/helpers.py'>

In [41]:
import glob

from pprint import pprint

import rasterio
import matplotlib.pyplot as plt

import numpy as np

from skimage import exposure


from skimage.io import imsave, imread

import os

from tqdm.notebook import tqdm



# Getting The Different Combination Of Bands, Indices or Properties And Converting TIF File Into JPG Format.

In [42]:
# define test name 
test_name = 'nir_green_blue'

test_folder = '../../data/' + test_name 

# define parameters
output_path = test_folder + '/images/'

# define paths with raster data
rasters_file_path = '../../data/source_data/*.tif'

# image specifics:
band_list = [7,2,1]
file_type = 'jpg' # jpg or png

# misc
plot_results = False

In [43]:
# find files in raster folder
raster_files = glob.glob(rasters_file_path)

print('nr files found: ', len(raster_files))

nr files found:  225


In [44]:
# create output directories if they do not exist
make_dir(test_folder)
make_dir(output_path)

Directory  ../../data/nir_green_blue  already exists...
Directory  ../../data/nir_green_blue/images/  already exists...


In [45]:
# loop over all files and write them to output folder
for raster_file in tqdm(raster_files):

    dataset = rasterio.open(raster_file)
    
    # read and reformat raster data
    img = dataset.read()
    bands, x, y = img.shape
    
    coastal = img[0].astype(np.float32)
    blue = img[1].astype(np.float32)
    green = img[2].astype(np.float32)
    red = img[4].astype(np.float32)
    red_edge = img[5].astype(np.float32)
    nir1 = img[6].astype(np.float32)
    nir2 = img[7].astype(np.float32)
    
    ndvi = reshape((nir1 - red)/(nir1 + red + 10e-5))
    ndwi = reshape((green - nir1) / (green + nir1 + 10e-5))
    wwi = reshape((coastal - nir2) / (coastal + nir2 + 10e-5))
    ccci = reshape((nir2 - red) / (nir2 + red + 10e-5))
    
    # use when we are combining bands or indices
    jpg = np.zeros((x,y,3))
    jpg[...,0] = ndwi
    jpg[...,1] = wwi
    jpg[...,2] = ccci
    
    # get image_id from filename
    stringlist = raster_file.split('/')[-1].split('_')
#     print(stringlist)
#     image_id = stringlist[-3]
    image_id = stringlist[-1].replace('.tif','')

    # get metadata 
#     record = gbdx.catalog.get(image_id)
#     offNadirAngle = record['properties']['offNadirAngle']
#     sunAzimuth = record['properties']['sunAzimuth']
    
    # used when we are enhancing the image as well as using the properties
    img_plot_raw = img[band_list,:,:]
#     img_plot_raw[0,:,:] = wwi
#     img_plot_raw[1,:,:] = offNadirAngle
#     img_plot_raw[2,:,:] = sunAzimuth
    img_plot = np.rot90(np.fliplr(img_plot_raw.T))
    
    # correct exposure for each band individually
    img_plot_enhance = np.array(img_plot, copy=True)
    
    
#     for band in range(3):
        
#         # check max amount of a single value
#         max_count_single_value = np.max(np.unique(img_plot, return_counts=True)[1])
        
#         # if there are more than specific values set them as nan
#         if max_count_single_value > max_single_value_count:
#             no_data_value = img_plot.flatten()[np.argmax(np.unique(img_plot, return_counts=True)[1])]
#             img_plot[img_plot == no_data_value] = np.nan 
            
#         p_1, p_2 = np.nanpercentile(img_plot[:,:,band], (lower_percentile, upper_percentile))
#         img_plot_enhance[:,:,band] = exposure.rescale_intensity(img_plot[:,:,band], 
#                                                             in_range=(p_1, p_2), 
#                                                             out_range = 'uint8')  
  

    img_plot_enhance = exposure.rescale_intensity(img_plot.T, 
                                                            in_range=(0,9000), 
                                                            out_range = (0,3))
    
    img_plot_enhance[img_plot_enhance>1] = 1
    
    img_plot_enhance = img_plot_enhance.T
    
    # name string 
    orig_file_name = raster_file.split('/')[-1].split('.')[0]
    
    # write jpg file
    file_path_jpg = '{0}{1}.{2}'.format(output_path,orig_file_name, file_type)
    file_name_jpg = 'example_{0}.{1}'.format(orig_file_name, file_type)
    imsave(file_path_jpg, img_plot_enhance)
    
    
    if plot_results:
        plt.imshow(img_plot_enhance)
        plt.title('Band used as [red, green, blue]: {}'.format(band_list))
        plt.show()
        
        print('--------------------------------------------')
        print('original file: \n', raster_file)
        print('--------------------------------------------')
        print('file written to: \n', file_path_jpg)
        print('--------------------------------------------')

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
